# Optimización de Esquema de calificación

Buscamos que estudiantes que han obtenido un rendimiento regular durante el curso no obtengan aprobatorio cuando no lo merecen.

In [1]:
import numpy as np
import pandas as pd
import copy
import time
import numpy as np
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', None)

## Configuración de la evaluación

En este diccionadio se configura cómo serán las evaluaciones.

In [2]:
config_evaluacion = {
    "regular_items": [
        # MÓDULO 1: 
        {
            "name": "Quiz_1", 
            "weakness": 3.5,
            "correlated_with": "Tarea_1", "correlation_factor": 0.5,
            "type": "easy",
            "threshold": 2.2 
        },
        {
            "name": "Tarea_1", 
            "weakness": 2.5,
            "min": 0.05, "max": 0.20, "suggested": 0.15, 
            "type": "advanced",
            "threshold": 3.0 
        },
        
        # MÓDULO 2: Este módulo tiene los temas más difíciles del curso (filter)
        {
            "name": "Quiz_2", 
            "correlated_with": "Quiz_1", "correlation_factor": 1.2,
            "weakness": 3.5,
            "type": "easy",
            "threshold": 2.5 
        },
        {
            "name": "Tarea_2", 
            "correlated_with": "Tarea_1", "correlation_factor": 1.2,
            "weakness": 2.5,
            "type": "filter",
            "threshold": 2.8 
        },
        
        # MÓDULO 3
        {
            "name": "Quiz_3", 
            "correlated_with": "Quiz_1", "correlation_factor": 1.0,
            "weakness": 3.5,
            "type": "easy",
            "threshold": 2.0 
        },
        {
            "name": "Tarea_3", 
            "weakness": 2.5,
            "correlated_with": "Tarea_1", "correlation_factor": 1.0,
            "type": "advanced",
            "threshold": 3.0 
        },
    ],

    "definitory_item": {
        "name": "Examen Final",
        "sudden_death": 2.50,
        "max_weight": 0.40,
        "type": "exam",
        "threshold": 2.5
    }
}

# AUTO-AJUSTE PARA CORRELACIONADOS
print("Autoconfigurando items correlacionados...")
reg_items = config_evaluacion['regular_items']
item_map = {item['name']: item for item in reg_items}

for item in reg_items:
    if 'correlated_with' in item:
        parent_name = item['correlated_with']
        factor = item['correlation_factor']
        if parent_name in item_map:
            parent = item_map[parent_name]
            item['peso_min'] = parent.get('peso_min', 0.0) * factor
            item['peso_max'] = parent.get('peso_max', 1.0) * factor
            item['peso_sugerido'] = parent.get('peso_sugerido', 0.0) * factor
            print(f" > {item['name']}: Ajustado vs {parent_name} (x{factor})")
        else:
            print(f" [!] Error: Padre '{parent_name}' no encontrado para '{item['name']}'")

Autoconfigurando items correlacionados...
 > Quiz_1: Ajustado vs Tarea_1 (x0.5)
 > Quiz_2: Ajustado vs Quiz_1 (x1.2)
 > Tarea_2: Ajustado vs Tarea_1 (x1.2)
 > Quiz_3: Ajustado vs Quiz_1 (x1.0)
 > Tarea_3: Ajustado vs Tarea_1 (x1.0)


In [3]:
config_evaluacion

{'regular_items': [{'name': 'Quiz_1',
   'weakness': 3.5,
   'correlated_with': 'Tarea_1',
   'correlation_factor': 0.5,
   'type': 'easy',
   'threshold': 2.2,
   'peso_min': 0.025,
   'peso_max': 0.1,
   'peso_sugerido': 0.075},
  {'name': 'Tarea_1',
   'weakness': 2.5,
   'peso_min': 0.05,
   'peso_max': 0.2,
   'peso_sugerido': 0.15,
   'type': 'advanced',
   'threshold': 3.0},
  {'name': 'Quiz_2',
   'correlated_with': 'Quiz_1',
   'correlation_factor': 1.2,
   'weakness': 3.5,
   'type': 'easy',
   'threshold': 2.5,
   'peso_min': 0.03,
   'peso_max': 0.12,
   'peso_sugerido': 0.09},
  {'name': 'Tarea_2',
   'correlated_with': 'Tarea_1',
   'correlation_factor': 1.2,
   'weakness': 2.5,
   'type': 'filter',
   'threshold': 2.8,
   'peso_min': 0.06,
   'peso_max': 0.24,
   'peso_sugerido': 0.18},
  {'name': 'Quiz_3',
   'correlated_with': 'Quiz_1',
   'correlation_factor': 1.0,
   'weakness': 3.5,
   'type': 'easy',
   'threshold': 2.0,
   'peso_min': 0.025,
   'peso_max': 0.1,
  

## Rutinas de ajustes de pesos

In [10]:
def get_independent_indices(config):
    indices = []
    items = config['regular_items']
    for i, item in enumerate(items):
        if 'correlated_with' not in item:
            indices.append(i)
    return indices

def reconstruct_full_weights(x_independent, config):
    items = config['regular_items']
    indep_indices = get_independent_indices(config)
    
    weight_map = {}
    
    # 1. Fill Independent
    current_indep_idx = 0
    full_weights = np.zeros(len(items))
    
    for i in indep_indices:
        val = x_independent[current_indep_idx]
        item_name = items[i]['name']
        weight_map[item_name] = val
        full_weights[i] = val
        current_indep_idx += 1
        
    # 3-pass loop for dependencies
    for _ in range(3):
        for i, item in enumerate(items):
            if full_weights[i] == 0.0 and 'correlated_with' in item:
                parent = item['correlated_with']
                if parent in weight_map:
                    w_val = weight_map[parent] * item['correlation_factor']
                    full_weights[i] = w_val
                    weight_map[item['name']] = w_val
                    
    return full_weights

def get_derived_weights(config_in):
    reg_items = config_in['regular_items']
    def_item = config_in['definitory_item']
    weight_map = {}
    
    # Init
    for item in reg_items:
        if 'weight' in item:
            weight_map[item['name']] = item['weight']
        elif 'correlated_with' not in item:
            weight_map[item['name']] = item.get('peso_sugerido', 0.0)
            
    # Propagate (Loop safe)
    final_reg_weights = []
    for _ in range(3): 
        for item in reg_items:
            w = 0.0
            if 'weight' in item:
                 w = item['weight']
            elif 'correlated_with' in item:
                 parent = item['correlated_with']
                 if parent in weight_map:
                     w = weight_map[parent] * item['correlation_factor']
                 else:
                     w = item.get('peso_sugerido', 0.0)
            else:
                 w = item.get('peso_sugerido', 0.0)
            weight_map[item['name']] = w
            
    # Final extraction
    final_reg_weights = [weight_map[item['name']] for item in reg_items]
    final_reg_weights = np.array(final_reg_weights)
    
    # Residual
    if 'weight' in def_item:
        ft_w = def_item['weight']
    else:
        ft_w = 1.0 - np.sum(final_reg_weights)
        if ft_w < 0: ft_w = 0.0
        
    return final_reg_weights, ft_w

## Rutinas de generación de datos de notas

Los tipos de distribución se toman con base en la experiencia en cursos STEM.

In [13]:
def generar_distribucion_bimodal(n, low_center=1.5, high_center=4.5, ratio=0.5, sigma=0.6):
    n_high = int(n * ratio)
    n_low = n - n_high
    high_grades = np.random.normal(high_center, sigma, n_high)
    low_grades = np.random.normal(low_center, sigma, n_low)
    combined = np.concatenate([high_grades, low_grades])
    np.random.shuffle(combined)
    return np.clip(combined, 0, 5)

def generar_distribucion_negative_skew(n, mode=4.5, sigma=1.0):
    raw = mode - np.random.exponential(scale=0.8, size=n)
    return np.clip(raw, 0, 5)

def generar_distribucion_normal(n, mean=3.0, sigma=1.0):
    return np.clip(np.random.normal(mean, sigma, n), 0, 5)

def generar_distribucion_exam(n, mean=2.8, sigma=1.1):
    return np.clip(np.random.normal(mean, sigma, n), 0, 5)

def generate_data(config, N=2000):
    data = {}
    
    def generate_column_by_type(dtype):
        if dtype == 'filter':
            return generar_distribucion_bimodal(N, low_center=1.2, high_center=4.0, ratio=0.4)
        elif dtype == 'easy':
            return generar_distribucion_negative_skew(N, mode=4.5)
        elif dtype == 'exam':
            return generar_distribucion_exam(N, mean=2.8, sigma=1.1)
        else: # advanced / normal
            return generar_distribucion_normal(N, mean=3.0, sigma=1.0)

    # Regular Items
    for item in config['regular_items']:
        dtype = item.get('type', 'advanced')
        data[item['name']] = generate_column_by_type(dtype)
        
    # Definitory Item
    def_item = config['definitory_item']
    dtype = def_item.get('type', 'advanced')
    data[def_item['name']] = generate_column_by_type(dtype)
    
    df = pd.DataFrame(data)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    return df.round(2)

# Ejemplo de generación de datos
df_notas = generate_data(config_evaluacion, N=1000)
print(f"Datos generados: {df_notas.shape}")

df_notas.head(10)

Datos generados: (1000, 7)


,Quiz_1,Tarea_1,Quiz_2,Tarea_2,Quiz_3,Tarea_3,Examen Final
0,4.31,3.03,4.40,4.28,3.84,1.88,2.80
1,3.99,4.83,3.38,1.21,4.35,3.12,1.12
2,3.54,1.85,3.78,0.42,3.91,1.25,4.76
3,3.20,5.00,2.96,3.62,3.83,3.15,2.74
4,1.52,3.97,4.47,3.93,3.82,2.05,3.26
5,1.91,2.68,4.02,3.83,3.69,3.09,2.38
6,3.58,2.72,3.20,1.22,4.12,2.34,5.00
7,3.64,2.61,4.02,1.03,0.78,3.98,4.20
8,1.96,2.56,4.16,0.84,4.04,2.70,1.88
9,1.10,1.42,4.29,1.16,4.32,4.17,4.16


## Generación de notas

In [ ]:
print(f"Generando notas (100 estudiantes, Rango 3.0-3.5...")

# 0. Force Fresh Randomness
np.random.seed(int(time.time()))

# 1. Generate Large Dataset
df_survey = generate_data(config_evaluacion, N=5000)

# Translation Map
type_map = {
    'easy': 'fácil',
    'advanced': 'avanzado',
    'filter': 'clave',
    'exam': 'examen'
}

# 2. Derive Suggested Weights & Headers
cols_ordered = []
weights_ordered = []
headers = []

reg_weights, w_final = get_derived_weights(config_evaluacion)
reg_items = config_evaluacion['regular_items']

# Build columns and FULL weights (reg + final) for Avg
for i, item in enumerate(reg_items):
    w_p = reg_weights[i] * 100.0
    t_orig = item.get('type','?')
    t_trans = type_map.get(t_orig, t_orig)
    header_str = f"{item['name']} [{w_p:.1f}%, {t_trans}]"
    cols_ordered.append(item['name'])
    weights_ordered.append(reg_weights[i])
    headers.append(header_str)

def_item = config_evaluacion['definitory_item']
w_f_p = w_final * 100.0
t_orig = def_item.get('type','?')
t_trans = type_map.get(t_orig, t_orig)
header_str = f"{def_item['name']} [{w_f_p:.1f}%, {t_trans}]"
cols_ordered.append(def_item['name'])
weights_ordered.append(w_final)
headers.append(header_str)

# 3. Calculate Average (Internal for filtering)
vals = df_survey[cols_ordered].values
avg = np.dot(vals, np.array(weights_ordered))
df_survey['Promedio'] = avg

# 4. Filter 3.0 - 3.5
mask_survey = (df_survey['Promedio'] >= 3.0) & (df_survey['Promedio'] <= 3.5)
df_filtered = df_survey[mask_survey].copy()

# 5. Sample 100
if len(df_filtered) > 100:
    df_sample = df_filtered.sample(n=100)
else:
    df_sample = df_filtered

print(f"Estudiantes encontrados en rango: {len(df_filtered)}. Seleccionados: {len(df_sample)}")

# 6. Format Final
df_final = df_sample[cols_ordered].round(1)
df_final.columns = headers

# 7. Add Judgment Column (Conditional)
col_name = '¿Debería Ganar? (1=Sí, 0=No)'
df_final[col_name] = ""

# 8. Save
timestamp = time.strftime("%Y%m%d_%H%M%S")
out_file = f"encuesta_profesores_{timestamp}.xlsx"
df_final.to_excel(out_file, index=False)
print(f"Archivo guardado: {out_file}")
display(df_final.head())

Generando notas (100 estudiantes, Rango 3.0-3.5...
Estudiantes encontrados en rango: 1698. Seleccionados: 100
Archivo guardado: encuesta_profesores_20260201_093259.xlsx


,"Quiz_1 [7.5%, fácil]","Tarea_1 [15.0%, avanzado]","Quiz_2 [9.0%, fácil]","Tarea_2 [18.0%, clave]","Quiz_3 [7.5%, fácil]","Tarea_3 [15.0%, avanzado]","Examen Final [28.0%, examen]","¿Debería Ganar? (1=Sí, 0=No)"
1114,4.10,1.20,2.30,4.70,1.80,4.60,3.50,
4871,3.70,3.20,4.10,4.00,3.20,3.70,3.00,
4467,3.20,2.70,4.20,0.50,4.40,3.90,3.50,
2922,3.80,1.10,3.20,4.00,4.40,3.90,4.00,
2119,4.40,2.20,3.60,1.20,4.40,3.20,3.60,
